In [1]:
import pandas as pd
train_df = pd.read_csv('C:/Users/USER/Desktop/Data_Practices/Disaster_Tweeter/train.csv')
test_df = pd.read_csv('C:/Users/USER/Desktop/Data_Practices/Disaster_Tweeter/test.csv')

## 훈련 데이터 탐색

In [2]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
# 키워드에 대한 변수 확인
train_df.keyword.unique()[:10]
# 키워드에 따라 어떤 정보가 주어져있을 가능성이 매우 높다.

array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon'], dtype=object)

In [4]:
# target에 대한 정보 확인
train_df.target.value_counts()
# 재난 트윗 3271건, 비재난 트윗 4342건 발생
# 주어진 데이터는 크게 불균형이 발생하지 않았다.

0    4342
1    3271
Name: target, dtype: int64

## 문자열 정제
다음과 같은 정제가 필요할 것이다.  
1. 영어이므로 대소문자를 통일한다.
2. 특수기호 등을 없애줘야 한다.  
3. 의미 없는 문법에 의한 단어들(stopwords)를 제거한다.

이 중 stop_words의 경우에는, tf-idf vectorizer 과정에서 지정가능하므로
나머지 과정에 대해 처리한다.

In [5]:
import re
import string
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) # 괄호 제거
    text = re.sub('https?://\S+|www\.\S+', '', text) # 링크 제거
    text = re.sub('<.*?>+', '', text) 
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #구두점 제거
    text = re.sub('\n', '', text) # 
    text = re.sub('\w*\d\w*', '', text)
    return text

In [6]:
train_df.text = train_df.text.apply(lambda s : clean_text(s))
test_df.text = test_df.text.apply(lambda s : clean_text(s))

In [7]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,people receive wildfires evacuation orders in...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,ariaahrary thetawniest the out of control wild...,1
7610,10871,NaN,NaN,s of volcano hawaii,1
7611,10872,NaN,NaN,police investigating after an ebike collided w...,1


In [8]:
train_df = train_df.fillna('.')
test_df = test_df.fillna('.')

## Vectorizer 적용 후의 간단한 분류 모델 평가

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english')
train_vec = tfidf.fit_transform(train_df.text)
test_vec = tfidf.transform(test_df.text)

In [10]:
train_vec.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
clf_nb = MultinomialNB()
scores = cross_val_score(clf_nb, train_vec, train_df['target'], cv = 5,
                        scoring = 'f1')
print(scores)

[0.59398496 0.58263773 0.6460251  0.60493827 0.73635665]


## 모델 개선 시도

다음에 대해 고려하고 모델을 적용해보자.  
- 단어의 기본형만 사용할 수 없을 것인가?
- 다른 벡터화는 존재하지 않는가?
- 다른 정보를 사용할 수는 없을 것인가?

### lemmatizer(표제어 추출)
이 과정을 통해 단어를 표제어 기준의 형태로 변경해볼 수 있다.  
하지만 이를 위해서는 위의 형태로 진행하는 것이 아닌 토큰화를 별도로 진행해야 할 것이다.

In [12]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
def text_preprocess(text):
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    tokens = tokenizer.tokenize(text)
    lemmatize_result = " ".join(lemmatizer.lemmatize(token) for token in tokens)
    
    return lemmatize_result

In [14]:
train_df.text = train_df.text.apply(lambda s : text_preprocess(s))
test_df.text = test_df.text.apply(lambda s : text_preprocess(s))

In [15]:
train_df

,id,keyword,location,text,target
0,1,.,.,our deed are the reason of this earthquake may...,1
1,4,.,.,forest fire near la ronge sask canada,1
2,5,.,.,all resident asked to shelter in place are bei...,1
3,6,.,.,people receive wildfire evacuation order in ca...,1
4,7,.,.,just got sent this photo from ruby alaska a sm...,1
...,...,...,...,...,...
7608,10869,.,.,two giant crane holding a bridge collapse into...,1
7609,10870,.,.,ariaahrary thetawniest the out of control wild...,1
7610,10871,.,.,s of volcano hawaii,1
7611,10872,.,.,police investigating after an ebike collided w...,1


In [16]:
# 추가 전처리 후 동일 모델로 재평가
tfidf = TfidfVectorizer(stop_words = 'english')
train_vec = tfidf.fit_transform(train_df.text)
test_vec = tfidf.transform(test_df.text)

clf_nb = MultinomialNB()
scores = cross_val_score(clf_nb, train_vec, train_df['target'], cv = 5,
                        scoring = 'f1')
print(scores)

[0.59128823 0.60855263 0.64585045 0.61672474 0.74294432]


In [17]:
# 다른 vectorizer 활용
from sklearn.feature_extraction.text import CountVectorizer
cnt = CountVectorizer(stop_words = 'english')
train_vec = cnt.fit_transform(train_df.text)
test_vec = cnt.transform(test_df.text)

clf_nb = MultinomialNB()
scores = cross_val_score(clf_nb, train_vec, train_df['target'], cv = 5,
                        scoring = 'f1')
print(scores)

[0.62966031 0.60770328 0.65671642 0.66010598 0.73203492]


count vectorzier에서 더 좋은 성능을 보이고 있는데, 이는 재난 문자에서는 공통적인 키워드가 들어가 있을 것이기에, 그 공통적인 키워드의 빈출을 살리기 위해서는 문서별 특이어를 잡아주는 TF-IDF보다는 공통어에 치중된 CountVectorzier의 성능이 더 좋을 것이다.

In [18]:
# 결과 확인을 위한 작업
def submission(submission_file_path,model,test_vectors):
    sample_submission = pd.read_csv(submission_file_path)
    sample_submission["target"] = model.predict(test_vectors)
    sample_submission.to_csv("submission.csv", index=False)

In [19]:
submission_file_path = 'C:/Users/USER/Desktop/Data_Practices/Diaster_Tweeter/sample_submission.csv'
test_vectors=test_vec
model = clf_nb
model.fit(train_vec, train_df['target'])
submission(submission_file_path,clf_nb,test_vectors)

위의 모델은 0.79436이라는 결과를 얻었다. 이제 조금 더 좋은 결과를 얻기 위해 개선 작업을 진행하고자 한다.

### 추가 변수 탐색

In [20]:
# keyword가 있는 경우, 이미 keyword가 text에 포함되어 있음을 확인해 볼 수 있다.
train_df.loc[train_df.keyword != '.']

,id,keyword,location,text,target
31,48,ablaze,Birmingham,bbcmtd wholesale market ablaze,1
32,49,ablaze,Est. September 2012 - Bristol,we always try to bring the heavy metal rt,0
33,50,ablaze,AFRICA,africanbaze breaking newsnigeria flag set abla...,1
34,52,ablaze,"Philadelphia, PA",cry out for more set me ablaze,0
35,53,ablaze,"London, UK",on plus side look at the sky last night it wa ...,0
...,...,...,...,...,...
7578,10830,wrecked,.,cameronhacker and i wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",three day off from work and theyve pretty much...,0
7580,10832,wrecked,London,fx forex trading cramer igers word that wrecke...,0
7581,10833,wrecked,Lincoln,engineshed great atmosphere at the british lio...,0


### 다른 Word-Embedding : Word2Vec

In [21]:
!pip install gensim

Word2Vec을 위해서는 전체 코퍼스를 만들어야 하며,
또한 이를 위해서는 앞에처럼 CountVectorizer 단계에서 stopwords를 지정하는 것이 아닌, 사전에 이를 지정해서 제외해야 하고, 이를 위해서는 토큰화된 상황으로 시작해야한다.

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.corpus import stopwords

def text_preprocess(text):
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    tokens = tokenizer.tokenize(text)
    lemmatizes = [lemmatizer.lemmatize(token) for token in tokens]
    words = [w for w in lemmatizes if w not in stopwords.words('english')]    
    combined_word = ' '.join(w for w in words)
    
    return combined_word

In [24]:
train_df.text = train_df.text.apply(lambda s : text_preprocess(s))
test_df.text = test_df.text.apply(lambda s : text_preprocess(s))

In [25]:
# corpus 형성, 이 때 train, test 모두 해야 사이즈가 맞는다.
corpus = pd.concat([train_df, test_df]).text.values

In [26]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
corpus_token = [nltk.word_tokenize(word) for word in corpus]

In [28]:
from gensim.models import Word2Vec
word_vector_model = Word2Vec(sentences = corpus_token, vector_size = 150, 
                             window = 5, min_count = 5, sg = 1)

In [29]:
word_vector_model.wv.most_similar('help')

[('half', 0.9721816778182983),
 ('save', 0.9702348113059998),
 ('around', 0.9693827033042908),
 ('bush', 0.9689847826957703),
 ('danger', 0.9684366583824158),
 ('please', 0.9676346182823181),
 ('alarm', 0.9643679261207581),
 ('drought', 0.9634671211242676),
 ('whole', 0.9630415439605713),
 ('burn', 0.9625403881072998)]

In [30]:
# corpus에 대한 훈련 실시
word_vector_model.train(corpus_token, total_examples = len(corpus_token),
                       epochs = 32)

(2360880, 3143040)